In [1]:
import os

In [2]:
cwd = os.getcwd()
dataset_folder = "datasets"
target_dataset = "alzheimer"

In [3]:
dataset_path = os.path.join(cwd, dataset_folder,target_dataset)
dataset_group = os.listdir(dataset_path)
patient_group = os.listdir(os.path.join(dataset_path, dataset_group[1]))
n_patient_group = len(patient_group)

In [4]:
from PIL import Image
import numpy as np

In [5]:
dataset_group,patient_group

(['test', 'train'], ['AD', 'CONTROL', 'PD'])

In [6]:
from tqdm import tqdm

In [7]:
def extract(dataset_g, patient_g):
    
    print(dataset_group[dataset_g], patient_group[patient_g])

    filename = os.listdir(os.path.join(dataset_path,
                                       dataset_group[dataset_g],
                                       patient_group[patient_g]
                                       ))
    
    n_filename = len(filename)

    test_shape = np.array(Image.open(os.path.join(dataset_path,dataset_group[dataset_g],
                                    patient_group[patient_g], filename[0]))).shape
    
    DATASET = np.zeros((n_filename, test_shape[0]*test_shape[-1]))

    for f in tqdm(range(len(filename))):

        image = np.array(Image.open(
                                    os.path.join(
                                                dataset_path,
                                                dataset_group[dataset_g],
                                                patient_group[patient_g],
                                                filename[f]
                                                )
                                        ))

        if np.shape(image.shape)[0] == 3:
            image = image[:,:,0]
            
    DATASET[f,:] = image.reshape(test_shape[0]*test_shape[-1])
    
    return DATASET

In [8]:
dataset_group,patient_group

(['test', 'train'], ['AD', 'CONTROL', 'PD'])

In [9]:
train_AD = extract(dataset_g=1, patient_g=0)
print(train_AD.shape)
train_CTRL = extract(dataset_g=1, patient_g=1)
print(train_CTRL.shape)

train AD


  4%|▍         | 115/2561 [00:00<00:02, 1148.42it/s]

100%|██████████| 2561/2561 [00:02<00:00, 1133.92it/s]


(2561, 36608)
train CONTROL


100%|██████████| 3010/3010 [00:02<00:00, 1129.15it/s]

(3010, 36608)


In [19]:
train_AD.shape[-1]*train_AD.shape[0]

93753088

In [10]:
import pandas as pd

In [11]:
DATASET_AD = pd.melt(pd.DataFrame(train_AD), var_name="voxel", value_name="activity")
DATASET_AD = pd.concat([DATASET_AD, pd.DataFrame(np.zeros((DATASET_AD.shape[0])), columns=["patient"])], axis=1)

DATASET_CTRL = pd.melt(pd.DataFrame(train_CTRL), var_name="voxel", value_name="activity")
DATASET_CTRL = pd.concat([DATASET_CTRL, pd.DataFrame(np.ones((DATASET_CTRL.shape[0])), columns=["patient"])], axis=1)

In [12]:
DATASET_CTRL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110190080 entries, 0 to 110190079
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   voxel     int64  
 1   activity  float64
 2   patient   float64
dtypes: float64(2), int64(1)
memory usage: 2.5 GB


In [13]:
print(DATASET_CTRL.isnull().sum(), DATASET_CTRL.size)
print(DATASET_AD.isnull().sum(), DATASET_AD.size)

voxel       0
activity    0
patient     0
dtype: int64 330570240
voxel       0
activity    0
patient     0
dtype: int64 281259264


In [14]:
DATASET = pd.concat([DATASET_AD,DATASET_CTRL])
DATASET_AD, DATASET_CTRL = 0,0
DATASET.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203943168 entries, 0 to 110190079
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   voxel     int64  
 1   activity  float64
 2   patient   float64
dtypes: float64(2), int64(1)
memory usage: 6.1 GB


In [16]:
DATASET.nunique()

voxel       36608
activity      256
patient         2
dtype: int64